# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [11]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list += glob.glob(os.path.join(root,'*'))
    
#print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [20]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6987


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [1]:
# This should make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [2]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [3]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [4]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    session.execute("DROP TABLE IF EXISTS events_by_sessionid_iteminsession;")
    session.execute("""
    CREATE TABLE IF NOT EXISTS events_by_sessionid_iteminsession 
      ( 
         artist        TEXT, 
         iteminsession INT, 
         length        FLOAT, 
         sessionid     INT,
         song          TEXT,
         PRIMARY KEY(sessionid, iteminsession) 
      ); 
    """)
except Exception as e:
    print(e)


In [7]:
file = 'event_datafile_new.csv'
query = "INSERT INTO events_by_sessionid_iteminsession(artist, iteminsession, length, sessionid, song)"
query += " VALUES(%s, %s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
pd.set_option('display.width', 512) # to display complete row in one line
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM events_by_sessionid_iteminsession where sessionId = 338 AND itemInSession = 4;"
try:
    df = pd.DataFrame(list(session.execute(query)))
    print(df[["artist", "song", "length"]])
except Exception as e:
    print(e)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    session.execute("DROP TABLE IF EXISTS events_by_sessionid_userid;")
    session.execute("""
    CREATE TABLE IF NOT EXISTS events_by_sessionid_userid 
      ( 
         artist        TEXT,
         firstname     TEXT,
         iteminsession INT, 
         lastname      TEXT, 
         sessionid     INT,
         song          TEXT,
         userid        INT,
         PRIMARY KEY(sessionid, userid, iteminsession)
      );
    """)
except Exception as e:
    print(e)

                    

In [13]:
file = 'event_datafile_new.csv'
query = "INSERT INTO events_by_sessionid_userid(artist, firstname, iteminsession, lastname, sessionid, song, userid)"
query += " VALUES(%s, %s, %s, %s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10])))

In [23]:
pd.set_option('display.width', 512) # to display complete row in one line
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM events_by_sessionid_userid where userid = 10 AND sessionid = 182;"
try:
    df = pd.DataFrame(list(session.execute(query)))
    print(df[["artist", "song", "firstname", "lastname"]])
except Exception as e:
    print(e)

              artist                                               song firstname lastname
0   Down To The Bone                                 Keep On Keepin' On    Sylvie     Cruz
1       Three Drives                                        Greece 2000    Sylvie     Cruz
2  Sebastien Tellier                                          Kilometer    Sylvie     Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...    Sylvie     Cruz


In [27]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    session.execute("DROP TABLE IF EXISTS username_by_song;")
    session.execute("""
    CREATE TABLE IF NOT EXISTS username_by_song 
      ( 
         firstname     TEXT,
         lastname      TEXT, 
         song          TEXT,
         userid        TEXT,
         PRIMARY KEY(song, userid)
      );
    """)
except Exception as e:
    print(e)              

In [28]:
file = 'event_datafile_new.csv'
query = "INSERT INTO username_by_song(firstname, lastname, song, userid)"
query += " VALUES(%s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[1], line[4], line[9], line[10]))
        

In [29]:
pd.set_option('display.width', 512) # to display complete row in one line
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM username_by_song where song='All Hands Against His Own';"
try:
    df = pd.DataFrame(list(session.execute(query)))
    print(df[["firstname", "lastname"]])
except Exception as e:
    print(e)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [31]:
try:
    tables = ["events_by_sessionid_iteminsession", "events_by_sessionid_userid", "username_by_song"]
    for t in tables:
        session.execute(f"DROP TABLE IF EXISTS {t};")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()